# Entrega 1 - IIC2433
## Intregantes:
- Fernando Smith
- Borja Márquez
- Vicente Lavagnino

## 🌟 Contexto:

**Gokei** es una startup chilena cuyo propósito es simplificar la gestión de la salud para las personas, eliminando la burocracia y los procesos tediosos asociados a trámites como el agendamiento de horas médicas, la compra de medicamentos y, especialmente, la solicitud de reembolsos en isapres y seguros médicos. Gracias a su innovadora plataforma, Gokei ha logrado reducir significativamente los pasos necesarios para estos procesos, mejorando así la experiencia de sus usuarios.

Sin embargo, para seguir mejorando sus servicios y ofrecer una propuesta de **reembolso automático más precisa y confiable**, Gokei necesita procesar los datos existentes y construir un modelo que prediga si un reembolso será aprobado o no. Con esta herramienta, esperan anticipar los resultados de las solicitudes, optimizar el flujo de trabajo y aumentar la eficiencia de la plataforma.

Por ello, recurren a la ayuda de nuestro grupo, donde aplicaremos **conceptos de minería de datos** para procesar la información y desarrollar el modelo predictivo necesario. Esto permitirá a Gokei tomar decisiones más informadas y ofrecer un servicio de reembolso automático con mejores resultados.

## 📁 Conjunto de Datos

Para ello, contamos con información **anonimizada** de reembolsos realizados mediante la plataforma de Gokei, organizada en los siguientes archivos:

#### 🔸 `all-boleta-metadata.json`
Contiene los metadatos de las boletas médicas:
- `amount`: Monto de la boleta.
- `beneficiario`: ID único del beneficiario.
- `doctor_rut`: RUT del médico que emite la boleta.
- `gasto`: ID único del gasto asociado (referencia a `all-gastos.json`).
- `provider_rut`: RUT del prestador o institución.
- `receipt_id`: ID único de la boleta.
- `Creation Date`: Fecha de creación del registro de la boleta.
- `Modified Date`: Fecha de última modificación del registro de la boleta.
- `Slug`: Slug del registro.
- `Creator`: Creador del registro (ej. “App admin”).
- `unique id`: Identificador único del registro de boleta en el sistema.

#### 🔸 `all-gastos.json`
Recoge la información de los gastos médicos realizados:
- `beneficiario`: ID único del beneficiario (referencia a `beneficiarios.json`).
- `boleta_metadata`: Referencia a la boleta médica (puede estar vacío).
- `estado`: Estado actual del gasto (ej. “successful”, “failed” u otros).
- `etapa_rendicion`: Etapa del proceso de rendición del gasto.
- `evento`: Evento o programa al que corresponde el gasto.
- `fecha`: Fecha en que se realizó el gasto.
- `fecha_ultima_revision`: Fecha de última revisión del gasto (puede estar vacío).
- `gestionado_por`: Email de la persona que gestionó el gasto (puede estar vacío).
- `nombre`: Nombre asociado al gasto (puede estar vacío).
- `pagado_el`: Fecha de pago (puede estar vacío).
- `presentado_el`: Fecha de presentación del gasto (puede estar vacío).
- `rendicion_id`: ID de la rendición asociada (puede estar vacío).
- `rinde_isapre`: Indica si rinde por isapre (ej. “sí” o “no”).
- `rinde_seguro`: Indica si rinde por seguro complementario (ej. “sí” o “no”).
- `rinde_seguro2`: Indica si rinde por segundo seguro complementario (ej. “sí” o “no”).
- `subtotal_isapre`: Monto subtotal cubierto por isapre.
- `subtotal_isapre_was_discounted_from_rex`: Indica si el subtotal isapre fue descontado de rex (puede estar vacío).
- `subtotal_seguro`: Monto subtotal cubierto por el primer seguro complementario.
- `subtotal_seguro2`: Monto subtotal cubierto por el segundo seguro complementario (puede estar vacío).
- `tipo`: Tipo de gasto (ej. Fonoaudiología, Examen, Medicamento).
- `total`: Monto total del gasto.
- `usuario`: ID único del usuario que realizó el gasto (referencia a `user.json`).
- `Creation Date`: Fecha de creación del registro del gasto.
- `Modified Date`: Fecha de última modificación del registro del gasto.
- `Slug`: Slug del registro del gasto (puede estar vacío).
- `unique id`: Identificador único del gasto en el sistema.

#### 🔸 `beneficiario.json`
Incluye información detallada de los beneficiarios (usuarios titulares o cargas):
- `aux_initial_age`: Edad inicial registrada al momento del ingreso.
- `es_titular`: Indica si es el titular o carga (puede estar vacío).
- `fecha_de_nacimiento`: Fecha de nacimiento del beneficiario.
- `isapre`: Isapre a la que pertenece el beneficiario.
- `parentezco`: Relación con el titular (ej. Hijo, Cónyuge).
- `seguro`: Seguro complementario del beneficiario (puede estar vacío).
- `seguro2`: Segundo seguro complementario del beneficiario (puede estar vacío).
- `usuario`: ID único del usuario al que está asociado el beneficiario.
- `Creation Date`: Fecha de creación del registro del beneficiario.
- `Modified Date`: Fecha de última modificación del registro del beneficiario.
- `unique id`: Identificador único del beneficiario en el sistema.

#### 🔸 `user.json`
Contiene información general de los usuarios de la plataforma:
- `isapre`: Isapre a la que pertenece el usuario.
- `plan`: Identificador único del plan de salud del usuario.
- `seguro`: Seguro complementario del usuario (puede estar vacío).
- `seguro2`: Segundo seguro complementario del usuario (puede estar vacío).
- `subscription_date`: Fecha de suscripción (puede estar vacío).
- `Creation Date`: Fecha de creación del registro del usuario.
- `Modified Date`: Fecha de última modificación del registro del usuario.
- `unique id`: Identificador único del usuario en la plataforma.

--- 
## 🚀 Desafío 
A partir de estos datos, aplicaremos técnicas de minería de datos para:
- Analizar patrones.
- Identificar variables relevantes.
- Construir un modelo predictivo que permita anticipar la aprobación de los reembolsos y mejorar la propuesta de **reembolso automático**.

<img src="image-20250602-162708.png" width="" align="" />

### Preprocesamiento

Para poder tratar los datos de una manera adecuada al ejercicio, lo primero que haremos será crear un `.csv` único con todos los datos necesarios, haciendo que cada gasto sea una fila del dataset y cada columna una información relevante para el ejercicio.

#### 📊 Lógica de Selección de Campos y Estructura Final del CSV

El objetivo es crear un archivo CSV consolidado que contenga toda la información **relevante** y **necesaria** de los gastos, integrando datos de los cuatro archivos JSON disponibles. A continuación, se explica la **lógica utilizada para determinar qué campos se incluyen** y cómo se construye el CSV final.

#### 🔎 Criterios de inclusión de registros

- Se incluyen **solo los gastos** cuyo campo `estado` en `all-gastos.json` sea **"successful"** o **"failed"**.
- Se omiten todos los registros de gasto que tengan otro valor en el campo `estado`.


#### 🔗 Integración de datos y "fallbacks"

Para cada gasto, se recopilan los siguientes campos:

1️⃣ **Datos básicos del gasto** (de `all-gastos.json`):
- `unique_id`, `beneficiario`, `estado`, `etapa_rendicion`, `evento`, `rinde_isapre`, `rinde_seguro`, `rinde_seguro2`, `subtotal_isapre`, `subtotal_seguro`, `subtotal_seguro2`, `tipo`, `total`, `usuario`.
- Fechas de creación y modificación (`created_at`, `modified_at`).

2️⃣ **Fecha de creación final**:
- Se prioriza la fecha de creación de la boleta (`Creation Date` de `all-boleta-metadata.json`) si existe para ese gasto.
- Si no existe boleta asociada, se utiliza la fecha de creación del gasto (`Creation Date` de `all-gastos.json`).

3️⃣ **Datos de la boleta** (de `all-boleta-metadata.json`):
- `doctor_rut`, `provider_rut`.
- Se obtienen **solo si existe una boleta asociada** con el gasto (relación por `gasto` en boleta con `unique id` en gasto).

4️⃣ **Información del beneficiario** (de `beneficiario.json`) con fallback en `user.json`:
- `isapre`: Primero se toma el de `beneficiario`, y si no existe se toma el de `user`.
- `seguro` y `seguro2`: Primero se toman del `beneficiario`. Si no existen y **`rinde_seguro` es "sí"**, se usan los valores de `user`.
- Si no existe ni en `beneficiario` ni en `user`, se deja vacío.

5️⃣ **Datos de usuario** (de `user.json`):
- Se utilizan como **fuente de fallback** para `isapre`, `seguro` y `seguro2`.


#### 📄 Estructura final del CSV

El CSV final **`gastos_filtrados.csv`** contiene las siguientes columnas:

| Campo              | Descripción                                                                                          |
|--------------------|------------------------------------------------------------------------------------------------------|
| `unique_id`        | Identificador único del gasto.                                                                       |
| `beneficiario`     | ID único del beneficiario.                                                                           |
| `estado`           | Estado del gasto (`successful` o `failed`).                                                          |
| `etapa_rendicion`  | Etapa del proceso de rendición del gasto.                                                            |
| `evento`           | Evento o programa al que corresponde el gasto.                                                       |
| `created_at`       | Fecha de creación (prioridad a boleta si existe, si no la del gasto).                                |
| `modified_at`      | Fecha de última modificación del gasto.                                                              |
| `rinde_isapre`     | Indicador de cobertura por isapre.                                                                   |
| `rinde_seguro`     | Indicador de cobertura por seguro complementario.                                                    |
| `rinde_seguro2`    | Indicador de cobertura por segundo seguro complementario.                                            |
| `subtotal_isapre`  | Monto subtotal cubierto por isapre.                                                                  |
| `subtotal_seguro`  | Monto subtotal cubierto por el primer seguro complementario.                                          |
| `subtotal_seguro2` | Monto subtotal cubierto por el segundo seguro complementario.                                         |
| `tipo`             | Tipo de gasto (ej. Fonoaudiología, Medicamento).                                                     |
| `total`            | Monto total del gasto.                                                                               |
| `usuario`          | ID único del usuario que realizó el gasto.                                                           |
| `doctor_rut`       | RUT del médico que emite la boleta (solo si existe boleta asociada).                                  |
| `provider_rut`     | RUT del prestador o institución (solo si existe boleta asociada).                                     |
| `isapre`           | Isapre del beneficiario (o del usuario si no existe en beneficiario).                                 |
| `seguro`           | Primer seguro complementario (del beneficiario o del usuario si `rinde_seguro` es "sí").              |
| `seguro2`          | Segundo seguro complementario (del beneficiario o del usuario si `rinde_seguro` es "sí").             |


#### 🔍 Consideraciones adicionales

- **Campos vacíos**: Si algún campo no existe en los archivos originales, se completa con una cadena vacía `""` en el CSV.
- **Relaciones clave**:
  - Boleta ↔ Gasto: `gasto` en boleta == `unique id` en gasto.
  - Gasto ↔ Beneficiario: `beneficiario` en gasto == `unique id` en beneficiario.
  - Gasto ↔ Usuario: `usuario` en gasto == `unique id` en usuario.


✅ Con esta lógica, el CSV resultante **consolida todos los datos relevantes** y permite análisis posteriores como validaciones, visualizaciones o integración con otras herramientas de análisis.  

In [2]:
import json
import csv

# Carga de datos
with open('all-boleta-metadata.json') as f:
    boletas_metadata = json.load(f)

with open('all-gastos.json') as f:
    gastos = json.load(f)

with open('beneficiarios.json') as f:
    beneficiarios = json.load(f)

with open('users.json') as f:
    usuarios = json.load(f)

# Índices rápidos
boleta_by_gasto = {b['gasto']: b for b in boletas_metadata}
beneficiario_by_id = {b['unique id']: b for b in beneficiarios}
usuario_by_id = {u['unique id']: u for u in usuarios}

# Archivo de salida
with open('gastos_filtrados.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = [
        'unique_id', 'beneficiario', 'estado', 'etapa_rendicion', 'evento',
        'created_at', 'modified_at', 'rinde_isapre', 'rinde_seguro', 'rinde_seguro2',
        'subtotal_isapre', 'subtotal_seguro', 'subtotal_seguro2', 'tipo', 'total',
        'usuario', 'doctor_rut', 'provider_rut', 'isapre', 'seguro', 'seguro2'
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for gasto in gastos:
        estado = gasto.get('estado', '').lower()
        if estado not in ['successful', 'failed']:
            continue  # Solo estados 'successful' o 'failed'

        # Buscar boleta asociada
        boleta = boleta_by_gasto.get(gasto['unique id'], {})

        # Buscar beneficiario y usuario asociados
        beneficiario = beneficiario_by_id.get(gasto.get('beneficiario', ''), {})
        usuario = usuario_by_id.get(gasto.get('usuario', ''), {})

        # created_at: prioridad boleta_metadata
        created_at = boleta.get('Creation Date') or gasto.get('Creation Date', '')

        # modified_at
        modified_at = gasto.get('Modified Date', '')

        # Fallbacks para isapre
        isapre = beneficiario.get('isapre', '') or usuario.get('isapre', '')

        # Fallbacks para seguro y seguro2 (si rinde_seguro == 'sí')
        rinde_seguro = gasto.get('rinde_seguro', '')
        seguro = beneficiario.get('seguro', '') or (usuario.get('seguro', '') if rinde_seguro == 'sí' else '')
        seguro2 = beneficiario.get('seguro2', '') or (usuario.get('seguro2', '') if rinde_seguro == 'sí' else '')

        # doctor_rut y provider_rut
        doctor_rut = boleta.get('doctor_rut', '')
        provider_rut = boleta.get('provider_rut', '')

        # Fila completa
        row = {
            'unique_id': gasto.get('unique id', ''),
            'beneficiario': gasto.get('beneficiario', ''),
            'estado': gasto.get('estado', ''),
            'etapa_rendicion': gasto.get('etapa_rendicion', ''),
            'evento': gasto.get('evento', ''),
            'created_at': created_at,
            'modified_at': modified_at,
            'rinde_isapre': gasto.get('rinde_isapre', ''),
            'rinde_seguro': rinde_seguro,
            'rinde_seguro2': gasto.get('rinde_seguro2', ''),
            'subtotal_isapre': gasto.get('subtotal_isapre', ''),
            'subtotal_seguro': gasto.get('subtotal_seguro', ''),
            'subtotal_seguro2': gasto.get('subtotal_seguro2', ''),
            'tipo': gasto.get('tipo', ''),
            'total': gasto.get('total', ''),
            'usuario': gasto.get('usuario', ''),
            'doctor_rut': doctor_rut,
            'provider_rut': provider_rut,
            'isapre': isapre,
            'seguro': seguro,
            'seguro2': seguro2
        }

        # Escribir la fila en el CSV
        writer.writerow(row)

print("✅ CSV creado correctamente: gastos_filtrados.csv")

✅ CSV creado correctamente: gastos_filtrados.csv
